In [ ]:
# reservoir pressure tracker
#
#
#
# last updated: 03/02/2020, v12
# - added colour groups to workbook to identify inputs and outputs
# - changed RDOL to latest version, split out a couple of RMUs for "injector RMUs"
# - MAJOR: added forecast and "matched" reservoir pressures based on material balance considerations for each RMU
#
# last updated: 08/01/2020, v11
# - MAJOR: Ability to add manual PBUs/PFOs to the automated table (use the Fusion sheet in the "Manual Additions" folder)
# - MAJOR: Now pulls all pressures at the last time point (to nearest hour) of a particular shut-in event, not just the final one
# - Alligin South wells (AP01, AW11) and RMU now inlcluded
# - New RDOL limits applied
# - Removed cached code, old inputs and reset custom Spark environment so should run more quickly
# - Hot-fix for Palantir dropping enumerated valve series (inc. hot fix for when they re-enumerated it again!)
# - Added in new "target pressure" band rather than the slew of PSAT lines we used to have
# - Finally fixed the "when is the original PBU/PFO shut in date" time stamp problem that has always bugged me but no-one noticed/cares about
#
# contact: D Robbins
#
# Functions defined here will be available to call in
# the code for any table.
import pandas as pd
import numpy as np
import math
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import copy
from textwrap import wrap
#
#####################
# RDOL plot options #
#####################
#
plot_well_annotations = 1 # if 0, omits the well annotations from RDOL plot
plot_dictionary = 1       # if 1, uses full RMU names rather than short-form
rdol_colour_map = ['red','gold','limegreen','gold','red'] # colour options for the RDOL stacked bars
plot_dictionary = 1   # if 0, plot the RMU acronyms (not recommended!)
rdol_plot_injectors = 0 # 1 = include injection wells in plots (flowing data)
rdol_latest = 1         # 1 = plot the most recent pressure points
rdol_IPC = 1            # 1 = plot the IPC
ylim_lower = 1300.0   # RDOL plot y-axis min
ylim_upper = 4000.0   # RDOL plot y-axis max
#
####################################
# PRES plotter variables to change #
####################################
#
pry_lower  = 1600.0   # PRES plot y-axis min
pry_upper  = 4000.0   # PRES plot y-axis min
#
# list of useful flags, integers and conversions
# LEAVE THESE ALONE (unless you know what you're doing)
bar_to_psi = 14.5037743897283 # (float)
m3hr_to_bpd = 150.9552 # (float)
lower_clip = 1000.0 # clips PRES (lower bound)
upper_clip = 5000.0 # clips PRES (upper bound)
#
####################################
# Longer shut-in detection options #
####################################
#
add_longer_shutins = 1 # 1 = include long shut-in times, 0 = just report requested values (int64 please)
longer_si_len = 672.0 # length of time (hours) for longer build-ups
longer_shutin_months = 12    # number of extra months to pull (int64)
#
####################################
# Plotter variables to leave alone #
####################################
#
upper_buffer = 1000.0 # additional 'redspace' at bottom of plot (arbitrarily high as plot will clip on ylim_upper)
ha_label = ['left','right']
va_label = ['bottom','top']
jitter_magnitude = 5     # potential magnitude of jitter from x-axis in pressure scatter plot
jitter_strength = 0.05   # strength of the swing
t_last_plot_inj = 30.0 # number of days required to plot last flowing pressure
#
#
###################################
# Global variables to leave alone #
###################################
#
pres_type_standard = 0   # normal t+dt point (consistent)
pres_type_longer   = 1   # recorded pressure point at 4, 8, 12, ... weeks (defined above)
pres_type_final    = 2   # last shut-in pressure, if well is offline at the current time that the PRES tracker is being updated
pres_type_manual   = 3   # points manually entered in Fusion sheet (GL \ Reservoir Pressure Tracker \ Manual additions \ Manual PRES entry)
pres_type_lastpoint = 4  # last pressure for each shut-in interval
#
i_online = 1
i_offline = 0
#
################
# Global lists #
################
#
# all examples, fill with data
list_of_RMUs = ['RMU1']
#
RMU_dictionary = {'RMU1':'My first RMU'}
#
#######################
# Global dictionaries #
#######################
#
# WELL DICTIONARY dict(key)[well_dict_*]
#
i_dict_well_topcomp = 0        # dP to top completion (Pgauge + this = Ptopcomp)
i_dict_well_datum = 1          # dP to datum          (Pgauge + this = Pdatum)
i_dict_well_gauge = 2          # reference gauge
i_dict_well_shutin = 3         # shut-in length
i_dict_well_rmu = 4  		   # RMU
i_dict_well_type = 5           # well type
i_dict_well_fric = 6		   # friction coefficients (FIELD units)
i_dict_well_correct = 7        # PRES correction (a.ln(t) + b - units of hr/bar) - [a,b,tmax]
#
# note injectors don't yet have PRES correction profiles, as I am lazy and producers are better PRES trackers
d_wells = {
                        #
						'P01':[176.59,72.77,'BHP',48.0,'S2E','prod',[0.000e0,0.000e0],[-5.27,28.0,200.0]],
						'I01':[2205.78,2236.52,'WHP',120.0,'S4W','inj',[1.591e-07,4.890e-04],[0.0,0.0,200.0]],
                       }
#
#
# WELLS-PER-RMU DICTIONARY (dict(key)[i_rmuwell_dict_*][list of n_wells/limits)
#
i_dict_rmuwell_wells = 0	# list of wells in RMU
i_dict_rmuwell_RDOL  = 1    # list of RMU limits [LSOL,LNOL,UNOL.USOL]
i_dict_LSOL = 0
i_dict_LNOL = 1
i_dict_UNOL = 2
i_dict_USOL = 3
#
d_RMU = {
    				   'RMU1':[['P01','P02','I01','I02'],[1453.0,1801.0,3201.0,3600.0]],
					   }
#
# RMU PRESSURE PLOTTER DICTIONARY dict(key)[list of n_lims][i_dict_plot_*]
# n_lims order: lnol, initial psat, other psats
#
i_dict_plotter_lnol = 0			# LNOL limit list
i_dict_plotter_psat0 = 1		# initial PSAT list
i_dict_plotter_psat = 2			# current PSAT list
i_dict_plotter_psat2 = 3		# current PSAT list #2
i_dict_plotter_value = 0        # limit short name
i_dict_plotter_value = 1        # limit plot value
i_dict_plotter_colour = 2       # limit plot colour
i_dict_plotter_linestyle = 3    # limit plot linestyle
i_dict_plotter_label = 4        # limit plot label
#
# RMU plotter for lines - make sure any satuaration ones begin with "psat" (short)
d_RMU_plotter_dictionary = {
							  'RMU1':[['lnol',1801.0,'red','solid','LNOL'],['psat_0',2729.0,'orchid','solid','Initial PSAT'],['psat',1577.0,'purple','dashed','Updated PSAT']],
}
#
# RMU plotter for optimum operation point: format is RMU:[lower_optimum - optimum - upper_optimum]
# (these numbers are in the RDOL picks spreadsheet in the RDOL area:
# \\Reuabzss001\resources\Reservoirs\Deepwater\Schiehallion\03 FINAL PRODUCTS\02 RPP\4.0 Reservoir Design Operating Limits\2019\RDOL refresh\RDOL picks)
i_optimum_low = 0
i_optimum_mid = 1
i_optimum_high = 2
#
d_optimum_operating_point = {
							  'RMU1':[1750.0,2000.0,2200.0],
}
#
# valve value-to-number dictionary (to fix the Palantir "feature" introduced on enumerated data series)
d_valve_enumerate = {
    "CLOSED":[66364536],    # also takes 0, don't see this on GL valves
    "OPEN":[-1010875264],   # also takes 1, don't see this on GL valves
    "Closed":[66364536],    # also takes 0, don't see this on GL valves
    "Open":[-1010875264],   # also takes 1, don't see this on GL valves
    "closed":[66364536],    # also takes 0, don't see this on GL valves
    "open":[-1010875264],   # also takes 1, don't see this on GL valves
    "UNDEFINED":[255],
    "Undefined":[-334512128],
    "undefined":[-334512128],
    }
#
#function to create labels above bars
def add_value_labels(annotation_label_size,ax, spacing=5):
    """Add labels to the end of each bar in a bar chart.

    Arguments:
        ax (matplotlib.axes.Axes): The matplotlib object containing the axes
            of the plot to annotate.
        spacing (int): The distance between the labels and the bars.
    """

    # For each bar: Place a label
    for rect in ax.patches:
        # Get X and Y placement of label from rect.
        y_value = rect.get_height()
        x_value = rect.get_x() + rect.get_width() / 2

        # Number of points between bar and label. Change to your liking.
        space = spacing
        # Vertical alignment for positive values
        va = 'bottom'

        # If value of bar is negative: Place label below bar
        if y_value < 0:
            # Invert space to place label below
            space *= -1
            # Vertically align label at top
            va = 'top'

        # Use Y value as label and format number with one decimal place
        label = "{:.1f}".format(y_value)

        # Create annotation
        ax.annotate(
            label,                      # Use `label` as label
            (x_value, y_value),         # Place label at end of the bar
            xytext=(0, space),          # Vertically shift label by `space`
            textcoords="offset points", # Interpret `xytext` as offset in points
            ha='center',                # Horizontally center label
            va=va,
            size=annotation_label_size)                      # Vertically align label differently for
                                        # positive and negative values.

In [ ]:
def RMU_list_df():

    # just return a DF with the RMU names in the global list as column titles, so we can use these as a template input
    df = pd.DataFrame([np.arange(len(list_of_RMUs))],columns=list_of_RMUs)
    
    return df

In [ ]:
def PRES_data_all(all_well_data, Manual_PRES_entry_PRES_manual_entry):
    #
    # script that loops over all the well data and extracts time and build-up/fall-off pressures
    # -> also extrapolates out to far-field pressure based on the a*ln(t)+b extrapolation PBU/PFO type curves (like a pbar ... call it pbar*)
    # -> also removes tubing friction component for the injectors
    #
    # contact: D Robbins, RE, WoS
    #
    # useful df column names to call
    well_dt_col_name = str('dt')                            # delta time between rows
    well_time_col_name = str('datetime')                        # timestamp

    # new dataframe to deep copy the data and leave the old df alone   
    df = all_well_data.copy(deep=True)
    df_manual = Manual_PRES_entry_PRES_manual_entry.copy(deep=True)

    # grab the well list from the dictionary
    well_list = d_well.keys()

    # add a column for delta t (in hours) between rows (used for all wells)
    df['dt'] = df['datetime'].diff().dt.seconds.div(3600,fill_value=0)

    # build requested list of longer shut-in times (used for all wells)
    longer_shutin_list = [i*longer_si_len for i in list(range(1,longer_shutin_months))] # list of extra months to pull (672 hr is 4 weeks for reference)

    # begin the beastly loop over all of the wells to grab the PBUs/PFOs
    #
    # temporary overwrite to just look at one well
    #well_list = ['WW04']
    for idf,well in enumerate(well_list):

        # iteration lists to build that will used for the dataframe
        list_of_slice_dates = []                # this will slice df on dates to report pbu/pfo pressure
        list_of_slice_types = []                # this will be a new column to describe if this is a short/long/final pressure
        list_of_PBU_PFO_dates = []              # this will be a new column stating the shut-in datetime for each pbu/pfo
        
        # useful column names 
        # columns in df
        well_gauge_col_name = str(well+'_'+d_well[well][i_dict_well_gauge]) # reference gauge
        well_MV_col_name = str(well+'_Master_valve')            # master valve
        well_WV_col_name = str(well+'_Wing_valve')              # wing valve
        well_rate_col_name = str(well+'_Inj_water_rate')       # preferred water injection rate (meter / choke CV)
        
        # columns in df_wells (this loop)
        well_status_col_name = str(well+'_status')              # well status (1/0)
        well_datum_pdatum_col_name = str(well+'_datum_psi')     # datumised pressure
        well_status_sum_col_name = str(well+'_shutin_time')     # total shutin time sum
        well_pbupfotype_col_name = str(well+'_shutin_type')     # shut in type (short, long, final)
        well_pbupfodate_col_name = str(well+'_pbu_pfo_date')    # shut-in date (NOT the same as reported pressure time)
        well_wellname_col_name = str('well_name')               # well name column

        # take subslice of pertinent well data in this loop - time, dt, BHP/WHP, master valve, wing valve,alloc water inj (if available)
        try: 
            df_well = df[[well_time_col_name,well_dt_col_name,well_gauge_col_name,well_MV_col_name,well_WV_col_name,well_rate_col_name]].copy(deep=True)
            # fill any NaNs in the allocated rates (otherwise the df is nuked by the .dropna() later)
            # have to fill with non-zero to prevent CW19 hanging on friction correction calc
            df_well[well_rate_col_name].fillna(0.0000001,inplace=True)
        except:
            df_well = df[[well_time_col_name,well_dt_col_name,well_gauge_col_name,well_MV_col_name,well_WV_col_name]].copy(deep=True)

        # do some df cleaning
        df_well = df_well.sort_values(by='datetime')
        df_well = df_well.dropna().reset_index(drop=True) # usually have some NaNs in the valve data ...
        
        # datumise the pressure and add this to the df_well
        df_well.loc[:,well_datum_pdatum_col_name] = df_well.loc[:,well_gauge_col_name]*bar_to_psi + d_well[well][i_dict_well_datum]

        # go through the valve states and report the well status (0 = offline, 1 = online)
        status = []         #   set up an empty list
        
        # for the mess that is the new Foundry valve enumeration, determine what state we have (integers or strings)
        my_valve_data_type = type(df_well[well_MV_col_name].iloc[0])
        if my_valve_data_type == str:
            i_valve_data_type = 1 # 1 = string based WV/MV description ('open' etc.)
        else:
            i_valve_data_type = 0 # 0 = long integer WV/MV description

        # loop over df and add a status 1 (online) or 0 (offline)
        # now using list comprehension for deliciously fast lööps
        if i_valve_data_type == 0:
            #df_well.loc[:,well_status_col_name] = [i_online if (MV<0 and WV<0) else i_offline for MV,WV in zip(df_well[well_MV_col_name],df_well[well_WV_col_name])]
            df_well.loc[:,well_status_col_name] = [i_online if (MV<0 and WV<0) else 0 for MV,WV in zip(df_well[well_MV_col_name],df_well[well_WV_col_name])]
        else:
            #df_well.loc[:,well_status_col_name] = [i_online if (MV == 'Open' and WV == 'Open') else i_offline for MV,WV in zip(df_well[well_MV_col_name],df_well[well_WV_col_name])]
            df_well.loc[:,well_status_col_name] = [i_online if (MV == 'Open' and WV == 'Open') else 0 for MV,WV in zip(df_well[well_MV_col_name],df_well[well_WV_col_name])]
        
        # sum up total OFFTIME and grab the start datetime of each PBU/PFO
        l_pbu_pfo_dates = []
        l_offline_length = []
        #
        d_shutin = df_well[well_time_col_name].iloc[0]      # PBU/PFO date, initialise with first value
        i_stat = 0
        #
        for i_offline,row in df_well.iterrows():
            # assume first row is offline, don't act on this
            if i_offline == 0:
                l_pbu_pfo_dates.append(d_shutin)
                l_offline_length.append(0.0)
            else:
                #
                # get well status and datetime
                row_stat = row[well_status_col_name]
                row_time = row[well_time_col_name]
                #
                # check if well is offline
                if row_stat == 0:
                    # (1) well was online prior to this - new shut-in, add to list of dates
                    if i_stat == 1:
                        # update time
                        d_shutin = row_time
                        # append last shut-in time and total shut-in length
                        l_pbu_pfo_dates.append(d_shutin)
                        l_offline_length.append(0.0)
                    #
                    # (2) well was offline prior to this
                    elif i_stat == 0:
                        # append last shut-in time and total shut-in length
                        l_pbu_pfo_dates.append(d_shutin)
                        l_offline_length.append(((row_time - d_shutin).total_seconds())/3600.0) # leave this as a float for now ...
                    #
                    # (3) unknown status, complain
                    else:
                        print(str('Warning: well '+well+' has unknown status at time '+str(row_time)))
                        # append last shut-in time and total shut-in length
                        l_pbu_pfo_dates.append(d_shutin)
                        l_offline_length.append(0.0)
                    #
                # if well online, just put in current time as placeholder
                else:
                    l_pbu_pfo_dates.append(row_time)
                    l_offline_length.append(0.0)
                #
                # update previous row status flag
                i_stat = row_stat
            #
        #
        # - first, build the list of requested shut-in times for this well, including longer shut-ins
        pull_shutin_dt = longer_shutin_list + [d_well[well][i_dict_well_shutin]]
        pull_shutin_dt.sort()   # shouldn't need a list = list.sort() style syntax
        #
        # loop over the offline length / pbu_pfo_dates list and build a list of wanted slice datetimes for the requested t+dt PBU/PFOs
        # we'll also build a list of the last recorded pressures for each pbu/pfo
        my_last_offline_length = l_offline_length[0]
        my_last_offline_date   = l_pbu_pfo_dates[0]
        my_last_datetime       = df_well[well_time_col_name].iloc[0]
        #
        for datetime,offline_length,offline_date in zip(df_well[well_time_col_name],l_offline_length,l_pbu_pfo_dates):
            #
            # check that the shut-in time is in the list of requested times by checking the offline length
            if offline_length in pull_shutin_dt:
                #
                # write "live" time to the date list
                list_of_slice_dates.append(datetime)
                #
                # write PBU/PFO start time to the list
                list_of_PBU_PFO_dates.append(offline_date)
                #
                # append the PRES type (short/long) to list of slice types
                if offline_length == d_well[well][i_dict_well_shutin]:
                    list_of_slice_types.append(pres_type_standard) # short-term pressure (requested dt; type 0)
                else:
                    list_of_slice_types.append(pres_type_longer)   # longer-term pressure (type 1)
                #
            #
            # check if the previous iteration point was the last point in the PBU/PFO
            # (this needs to be longer than the shortest period requested, or it's not considered to be suitable to report)
            # (also needs to NOT be in the existing list!)
            if (my_last_offline_length > offline_length) and (my_last_offline_length > d_well[well][i_dict_well_shutin]) and (my_last_datetime not in list_of_slice_dates):
                list_of_slice_types.append(pres_type_lastpoint)
                list_of_slice_dates.append(my_last_datetime)
                list_of_PBU_PFO_dates.append(my_last_offline_date)

            # remember the "last points"
            my_last_offline_length = offline_length
            my_last_offline_date = offline_date
            my_last_datetime = datetime

        # grab the last PBU/PFO pressure for the well
        # if d_well[well][i_dict_well_type] == 'prod':  # producer, report last shut-in pressure
        if len(df_well[df_well[well_status_col_name]==i_offline]) > 0:      # check the well has actually been offline
            # find the last date for which the well was shut in
            find_time = max(df_well[(df_well[well_status_col_name]==i_offline)]['datetime'].tolist())
            # report the time and PBU date for this final point
            found_dt = list_of_offline_length[df_well[well_time_col_name].tolist().index(find_time)]
            if (found_dt > d_well[well][i_dict_well_shutin]) and (find_time not in list_of_slice_dates): # make sure >48 hr and not already in list (by pure chance ...)
                list_of_slice_types.append(pres_type_final)
                list_of_slice_dates.append(find_time)
                list_of_PBU_PFO_dates.append(find_time)
        # else:                                                       # injector, report last flowing pressure
        #     if len(df_well[df_well[well_status_col_name]==i_online]) > 0:       # check the well has actually been online
        #         # find the last period for which the well was online
        #         find_time = max(df_well[(df_well[well_status_col_name]==i_online)]['datetime'].tolist())
        #         # report the pressure, time and PBU date for this final point IF it's within 30 days
        #         last_time = df_well.loc[len(df_well)-1,well_time_col_name]
        #         delta_t_found = ((last_time-find_time).total_seconds())/86400.0
        #         if delta_t_found < t_last_plot_inj:
        #             list_of_slice_types.append(pres_type_final)
        #             list_of_slice_dates.append(find_time)
        #             list_of_PBU_PFO_dates.append(find_time)
        
        #################################################################################################################
        # INSERT ANY SEGMENT/WELL SPECIFIC WELL OPTIONS HERE (e.g. gauge is null or not functional before/after a date) #
        #################################################################################################################

        # build an updated list of slice dates for CW26 as DHG failed until 20/10/18 restoration
        if well == 'CW26':
            new_slice_dates = []
            new_slice_types = []
            new_slice_pfodates = []
            for datetime,prestype,pfodate in zip(list_of_slice_dates,list_of_slice_types,list_of_PBU_PFO_dates):
                if datetime > pd.to_datetime('2018-10-20 15:00:00'):# ignore CW26 data up to here, DHG failed until here
                    new_slice_dates.append(datetime)
                    new_slice_types.append(prestype)
                    new_slice_pfodates.append(pfodate)
            # now update the original list
            list_of_slice_dates = new_slice_dates
            list_of_slice_types = new_slice_types
            list_of_PBU_PFO_dates = new_slice_pfodates
        
        # for CP07, gauge flatlined for the 2019-2020 new year so remove dates from this time
        if well == 'CP07':
            new_slice_dates = []
            new_slice_types = []
            new_slice_pfodates = []
            for datetime,prestype,pfodate in zip(list_of_slice_dates,list_of_slice_types,list_of_PBU_PFO_dates):
                # remove flatline period from data set (i.e. only keep dates NOT in this period)
                if (datetime < pd.to_datetime('2019-12-29 08:00:00')) or (datetime > pd.to_datetime('2020-01-01 02:15:00')):
                    new_slice_dates.append(datetime)
                    new_slice_types.append(prestype)
                    new_slice_pfodates.append(pfodate)
            # now update the original list
            list_of_slice_dates = new_slice_dates
            list_of_slice_types = new_slice_types
            list_of_PBU_PFO_dates = new_slice_pfodates
        
        # WP12 gauge flatlined April 2019, ignore these data (just so happens to fall on a t+dt period)
        if well == 'WP12':
            new_slice_dates = []
            new_slice_types = []
            new_slice_pfodates = []
            for datetime,prestype,pfodate in zip(list_of_slice_dates,list_of_slice_types,list_of_PBU_PFO_dates):
                # remove flatline period from data set (i.e. only keep dates NOT in this period)
                if (datetime < pd.to_datetime('2019-03-31 00:00:00')) or (datetime > pd.to_datetime('2019-04-18 00:00:00')):
                    new_slice_dates.append(datetime)
                    new_slice_types.append(prestype)
                    new_slice_pfodates.append(pfodate)
            # now update the original list
            list_of_slice_dates = new_slice_dates
            list_of_slice_types = new_slice_types
            list_of_PBU_PFO_dates = new_slice_pfodates

        # WW06 - gauge died Jan 2020 briefly
        if well == 'WW06':
            new_slice_dates = []
            new_slice_types = []
            new_slice_pfodates = []
            for datetime,prestype,pfodate in zip(list_of_slice_dates,list_of_slice_types,list_of_PBU_PFO_dates):
                # remove flatline period from data set (i.e. only keep dates NOT in this period)
                if (datetime < pd.to_datetime('2020-01-01 05:00:00')) or (datetime > pd.to_datetime('2020-01-06 14:00:00')):
                    new_slice_dates.append(datetime)
                    new_slice_types.append(prestype)
                    new_slice_pfodates.append(pfodate)
            # now update the original list
            list_of_slice_dates = new_slice_dates
            list_of_slice_types = new_slice_types
            list_of_PBU_PFO_dates = new_slice_pfodates

        ##################################
        # END SPECIFIC WELL OPTIONS HERE #
        ##################################
        
        # # for injectors, find the pres_type_final and add frictional correction
        # if d_well[well][i_dict_well_type] == 'inj':
        #     #
        #     if pres_type_final in list_of_slice_types:
        #         # get the date of the last flowing pressure point
        #         last_flowing_p_date = list_of_slice_dates[list_of_slice_types.index(pres_type_final)]
        #         # find the index of this date
        #         last_flowing_p_index = df_well[well_time_col_name].tolist().index(last_flowing_p_date)
        #         # get the first non-zero allocated rate prior to this
        #         final_rate = 0.0 # set up token value
        #         for i in range(last_flowing_p_index,0,-1):
        #             if df_well.iloc[i][well_rate_col_name] > 0.0:
        #                 final_rate = df_well.iloc[i][well_rate_col_name]*m3hr_to_bpd # gauge rate is metric ... should probably put some logic in here to check
        #         # now correct pressure for friction
        #         df_well.loc[last_flowing_p_index,well_datum_pdatum_col_name] = df_well.loc[last_flowing_p_index,well_datum_pdatum_col_name] - (final_rate*final_rate*d_well[well][i_dict_well_fric][0] + final_rate*d_well[well][i_dict_well_fric][1])
        #
        # slice the df on list of all the requested shut-in times for this well
        df_well = df_well[df_well[well_time_col_name].isin(list_of_slice_dates)]
        #
        # now add the columns of pressure types (short, long, final) and pbu_pfo start dates to this df
        df_well.loc[:,well_pbupfotype_col_name] = list_of_slice_types
        df_well.loc[:,well_pbupfodate_col_name] = list_of_PBU_PFO_dates
        df_well.loc[:,well_wellname_col_name] = [well]*len(df_well)
        #
        # take a slice of the df on only the columns wanted
        df_well = df_well[[well_wellname_col_name,well_time_col_name,well_datum_pdatum_col_name,well_pbupfodate_col_name,well_pbupfotype_col_name]]
        #
        # rename the column headers for the final reservoir pressure dataframe
        #
        df_well.columns = ['Well','time','Datum_PRES_psi','PBU_PFO_date','PBU_PFO_type']
        #
        # new and improved logic for concatenating data sets
        try:
            df_rmu_shutins = pd.concat([df_rmu_shutins,df_well],sort=False)
        except:
            df_rmu_shutins = df_well.copy(deep=True)
        #
        # print out some outputs ... be a little cute with the print nomenclature, probably won't work anywhere else other than Glen Lyon but hey ho
        if well[1] == 'P':        
            print(str('Well: '+str(well)+' PBUs added to database: '+str(len(df_well))))
        else:
            print(str('Well: '+str(well)+' PFOs added to database: '+str(len(df_well))))
        #
        # ... and loop to the next well!
    # (end of loop over wells)
    
    # clip the df for too outliers and NaN them out
    df_rmu_shutins['Datum_PRES_psi'] = df_rmu_shutins['Datum_PRES_psi'].clip(lower=lower_clip,upper=upper_clip).replace([lower_clip,upper_clip],np.nan)

    # drop the NaNs to get rid of the outliers
    df_rmu_shutins = df_rmu_shutins.dropna().sort_values(by='time')

    # correct the reservoir pressure using the logarithmic corrections
    corrected_PRES_list = []
    #
    for i,row in df_rmu_shutins.iterrows():
        #
        # grab some useful data from this row
        row_pbu_type = row['PBU_PFO_type']
        row_pbu_pres = row['Datum_PRES_psi'] 
        row_well_name = row['Well']
        row_well_type = d_well[row_well_name][i_dict_well_type]
        #
        # if well is a producer and PBU_PFO type is the initial one, determine correction
        if (row_well_type == 'prod') and (row_pbu_type == pres_type_standard):
            # get number of hours
            PBU_time = d_well[row_well_name][i_dict_well_shutin]
            # compute correction
            my_p_correction = (d_well[row_well_name][i_dict_well_correct][0]*math.log(PBU_time) + d_well[row_well_name][i_dict_well_correct][1])*bar_to_psi
            #
            corrected_PRES_list.append(row_pbu_pres+my_p_correction)
            #
        # otherwise, just use the quoted PRES as the "corrected" pressure
        else:
            corrected_PRES_list.append(row_pbu_pres)
    #
    # append list to df
    df_rmu_shutins.insert(len(df_rmu_shutins.columns),'Datum_PRES_psi_corrected',corrected_PRES_list)
    #
    # finally (honestly), call the manual entry df and add these values to the main shut-in DF
    #
    # first, empty list to build df
    l_manual_pbupfo = []
    #
    # loop over all wells and build manual entry df
    for well in well_list:
        #
        # check well has entry in the manual df
        if well in df_manual['Well'].unique():
            #
            # if it does, grab a sub-slices df for this well (could have more than one entry)
            df_well_manual = df_manual[df_manual['Well']==well]
            #
            # iterrate over each row (= 1 entry) and build appropriate column values
            for i_man,row_man in df_well_manual.iterrows():
                #
                # compute number of hours PBU / PFO pressure is specified at
                dt_shutin = ((row_man['PRES_time'] - row_man['PBU_PFO_start']).total_seconds())/3600.0
                #
                # determine datum pressure
                p_datum = row_man['P_gauge']*bar_to_psi + d_well[well][i_dict_well_datum]
                #
                # generate correction if length of shut in is shorter than the specified max time cap
                if dt_shutin < d_well[well][i_dict_well_correct][2]:
                    p_correction = (d_well[well][i_dict_well_correct][0]*math.log(dt_shutin) + d_well[well][i_dict_well_correct][1])
                else:
                    p_correction = 0.0
                #
                # add this manual shut in to the df list
                l_manual_pbupfo.append([well,row_man['PRES_time'],p_datum,row_man['PBU_PFO_start'],pres_type_manual,p_datum+p_correction])
                #
            #
        #
    #
    # now generate df from this table
    #df_manual_concat = pd.DataFrame(l_manual_pbupfo,columns=['Well','time','Datum_PRES_psi','PBU_PFO_date','PBU_PFO_type','Datum_PRES_psi_corrected'])
    #
    # concatenate the manual values onto the main df
    df_rmu_shutins = pd.concat([df_rmu_shutins,pd.DataFrame(l_manual_pbupfo,columns=['Well','time','Datum_PRES_psi','PBU_PFO_date','PBU_PFO_type','Datum_PRES_psi_corrected'])])
    #
    return df_rmu_shutins

In [ ]:
def RMU_pressures_plotter(RMU_list_df, PRES_data_all,Compressibility_history_match,Compressibility_forecasting):
    
     # set up plot options
    fig,ax = plt.subplots(figsize=(18,13))

    # grab df data
    df_plot = PRES_data_all.copy(deep=True)
    df_historical_voidage = Compressibility_history_match.copy(deep=True)
    df_forecast_voidage = Compressibility_forecasting.copy(deep=True)

    # specify the RMU wanted (we'll turn this into a template)
    RMU_wanted = 'S1NE'

    # other well options
    bool_plot_injectors = True
    bool_plot_short_only = False
    bool_plot_average_lines = False
    bool_show_optimum_window = True # if this is true, the PSAT lines will be ignored
    bool_plot_PSAT_lines = True    # effectively there for historical reasons
    bool_dynamic_pressure_limits = False # if true, compute the padded y limit from data 
    bool_plot_pressure = True       # if true, plot the average pressure line computed from MBAL
    bool_plot_forecast_pressure = False   # if true, show the estimated 30-day forecast pressure given current RMU strategy
    bool_plot_corrected = True   # if true, plot the corrected pressures

    # if we are showing the optimum window, don't bother with the PSAT lines as this will be included in the calculation
    if bool_show_optimum_window == True:
        bool_plot_PSAT_lines = False

    # set up the x_min time axis
    n_months_plot = int(60)
    t_max = pd.Timestamp.now() + pd.Timedelta(1,'M')
    t_min = pd.Timestamp.now() - pd.Timedelta(n_months_plot,'M')

    # define colour mapping; largest RMU at the moment has 12 wells, so set up a list of 15 just in case
    # see here for a good colour choice: https://matplotlib.org/examples/color/named_colors.html
    plt_colour_list = ['blue','mediumturquoise','darkgreen','darkviolet','darkgoldenrod','grey','gold','darkturquoise','darkgoldenrod','limegreen','saddlebrown','black','deeppink','greenyellow','darkmagenta']

    # grab the list of wells from this RMU
    # just plot producers for the second ...
    well_list = []
    for well in d_RMU_dictionary[RMU_wanted][i_dict_rmuwell_wells]:
        if bool_plot_injectors == False:
            if d_well[well][i_dict_well_type] == 'prod':
                well_list.append(well)
        else:
            well_list.append(well)

    # grab the limits for the RMU
    RMU_limits = d_RMU_plotter_dictionary[RMU_wanted]

    # slice df on wells, just return time and PRES
    if bool_plot_corrected == True:
        # change column name to save wordiness later on
        df_plot = df_plot[df_plot['Well'].isin(well_list)][['Well','time','Datum_PRES_psi_corrected','PBU_PFO_type']].rename(columns={'Datum_PRES_psi_corrected':'Datum_PRES_psi'})
    else:
        df_plot = df_plot[df_plot['Well'].isin(well_list)][['Well','time','Datum_PRES_psi','PBU_PFO_type']]

    if RMU_wanted == 'NWAD':
        gs = gridspec.GridSpec(3, 1, height_ratios=[4, 1, 1])
        gs.update(hspace=0.1)
    else:
        gs = gridspec.GridSpec(2, 1, height_ratios=[4, 1])
        gs.update(hspace=0.1)

    ax = plt.subplot(gs[0])
    
    # plot the pressures
    for i,well in enumerate(well_list):
        if bool_plot_short_only == True:
            ax.scatter(df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_standard)]['time'],df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_standard)]['Datum_PRES_psi'],label=well,c=plt_colour_list[i],marker='o')
        else:
            # short time (standard) shut-ins - filled circles
            ax.scatter(df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_standard)]['time'],df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_standard)]['Datum_PRES_psi'],label=well,c=plt_colour_list[i],marker='o')
            # longer time shut-ins - hollow circles
            ax.scatter(df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_longer)]['time'],df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_longer)]['Datum_PRES_psi'],marker='o',facecolors='none',edgecolors=plt_colour_list[i],label='_nolegend_')
            # final-time shut-ins - crosses
            ax.scatter(df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_final)]['time'],df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_final)]['Datum_PRES_psi'],marker='x',c=plt_colour_list[i],label='_nolegend_')
            # manual shut ins - plusses
            ax.scatter(df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_manual)]['time'],df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_manual)]['Datum_PRES_psi'],marker='+',c=plt_colour_list[i],label='_nolegend_')

    # plot moving averages
    if bool_plot_average_lines == True:
        for i,well in enumerate(well_list):
            # sub-slice df for the well and PBU/PFO types requested
            if bool_plot_short_only == True:
                df_ave_plot = df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_standard)]
            else:
                df_ave_plot = df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']!=pres_type_final)&(df_plot['PBU_PFO_type']!=pres_type_lastpoint)]
            #
            # grab list of times and sort
            df_ave_plot = df_ave_plot.sort_values(by='time')
            # generate rolling average (3 period)
            df_ave_plot.loc[:,'pres_ave'] = df_ave_plot.loc[:,'Datum_PRES_psi'].rolling(window=3,center=True).mean()
            # fill NaNs with initial/final values
            df_ave_plot['pres_ave'] = df_ave_plot['pres_ave'].fillna(df_ave_plot['Datum_PRES_psi'])
            # plot rolling ave on plot
            ax.plot(df_ave_plot['time'],df_ave_plot['pres_ave'],label='_nolegend_',c=plt_colour_list[i])

    # sub-slice the df so we don't include data (in the limits calc) prior to the requested interval 
    # (don't need to do max time as by default it's in the future so won't have any data for this condition ...)
    # (do this after the average lines so there is continuity in the plot)
    df_plot = df_plot[(df_plot['time'] >= t_min)]

    # compute the max/min pressures
    if bool_dynamic_pressure_limits == True:
        if bool_plot_short_only == True:
            y_max = max(df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_standard)]['Datum_PRES_psi'])
            y_min = min(df_plot[(df_plot['Well']==well)&(df_plot['PBU_PFO_type']==pres_type_standard)]['Datum_PRES_psi'])
        else:
            y_max = max(df_plot[df_plot['Well']==well]['Datum_PRES_psi'])
            y_min = min(df_plot[df_plot['Well']==well]['Datum_PRES_psi'])
        #
        # add 15% padding
        y_min *= 0.85
        y_max *= 1.15
        #
    else:
        # fallback options specified in the global code
        y_max = pry_upper
        y_min = pry_lower

    # get the min/max times for plot limit
    plot_t_min = pd.to_datetime(t_min)
    plot_t_max = pd.to_datetime(t_max)

    # if requested, plot the saturation pressure lines(old option)
    if bool_plot_PSAT_lines == True:

        # grab the limits for the RMU
        RMU_limits = d_RMU_plotter_dictionary[RMU_wanted]

        # plot the limits
        for limit in RMU_limits:
            ax.plot([plot_t_min,plot_t_max],2*[limit[i_dict_plotter_value]],linestyle=limit[i_dict_plotter_linestyle],c=limit[i_dict_plotter_colour],label=limit[i_dict_plotter_label],zorder=2)

    # if requested, plot the optimum set point and shade the window (new option)
    if bool_show_optimum_window == True:
        
        # grab the optimum set points
        RMU_optimum_set_points = d_optimum_operating_point[RMU_wanted]

        # plot the median line
        ax.plot([plot_t_min,plot_t_max],2*[RMU_optimum_set_points[i_optimum_mid]],linestyle='dashed',c='forestgreen',label='RMU target set point',zorder=2)

        # plot the upper/lower bounds as filled envelope
        ax.fill_between([plot_t_min,plot_t_max],2*[RMU_optimum_set_points[i_optimum_low]],2*[RMU_optimum_set_points[i_optimum_high]],label='_nolabel_',alpha=0.1,facecolor='forestgreen')

        # plot LNOL / UNOL if axis limit allows
        if y_min < d_RMU_dictionary[RMU_wanted][i_dict_rmuwell_RDOL][i_dict_LNOL]:
            ax.plot([plot_t_min,plot_t_max],2*[d_RMU_dictionary[RMU_wanted][i_dict_rmuwell_RDOL][i_dict_LNOL]],linestyle='solid',c='darkorange',label='LNOL',zorder=2)
        if y_max > d_RMU_dictionary[RMU_wanted][i_dict_rmuwell_RDOL][i_dict_UNOL]:
            ax.plot([plot_t_min,plot_t_max],2*[d_RMU_dictionary[RMU_wanted][i_dict_rmuwell_RDOL][i_dict_UNOL]],linestyle='solid',c='sandybrown',label='UNOL',zorder=2)

    # if requested, plot the MBAL pressure
    if bool_plot_pressure == True:
        #
        # don't have a MBAL model for both tanks, ignore this request
        if RMU_wanted != 'NWAD': 
            #
            # get the rmu df
            df_rmu_hp = df_historical_voidage[df_historical_voidage['rmu']==RMU_wanted].sort_values(by='time')
            df_rmu_fp = df_forecast_voidage[df_forecast_voidage['rmu']==RMU_wanted].sort_values(by='time')
            #
            # plot the pressures
            ax.plot(df_rmu_hp['time'],df_rmu_hp['estimated_PRES'],c='black',linestyle='solid',label='MBAL matched pressure')
            #
            if bool_plot_forecast_pressure == True:
                ax.plot(df_rmu_fp['time'],df_rmu_fp['forecast_P_est'],c='black',linestyle='dashed',label='MBAL forecast pressure')
            #
        #
    #

    # format the y axis
    plt.ylabel('Datumised pressure (psia)')
    plt.ylim(y_min,y_max)

    # gridlines to make extrapolation easier
    plt.grid(b=True, which='major', axis='both', color='grey', linestyle='-', alpha = 0.5)

    # set the legend to make plot visible
    plt.legend(loc='center left', bbox_to_anchor=(1.03, 0.5), ncol=1)

    # title
    plt.title(RMU_dictionary[RMU_wanted]+' reservoir pressures',fontsize='x-large',y=1.02)
    
    # adjust the subplot so the legend is still visible
    plt.subplots_adjust(right=0.75,bottom=0.14,left=0.1)

    if RMU_wanted == 'NWAD':
        df1 = df_historical_voidage[df_historical_voidage['rmu'] == 'NWADS2']
        df2 = df_historical_voidage[df_historical_voidage['rmu'] == 'NWADS3']

        ax2 = plt.subplot(gs[1], sharex = ax)
        ax2.plot(df1['time'],df1['cumulative_voidage'],c='black')
        ax2.plot(df1['time'],df1['cumulative_aquifer'],c='blue')
        plt.grid(b=True, which='major', axis='both', color='grey', linestyle='-', alpha = 0.5)

        ax3 = plt.subplot(gs[2], sharex = ax)
        ax3.plot(df2['time'],df2['cumulative_voidage'],c='black')
        ax3.plot(df2['time'],df2['cumulative_aquifer'],c='blue')
        plt.grid(b=True, which='major', axis='both', color='grey', linestyle='-', alpha = 0.5)

        # add label for y axis
        plt.ylabel('Cumulative offtake/aquifer (mmrb)')

    else:
        df = df_historical_voidage[df_historical_voidage['rmu'] == RMU_wanted]
        ax2 = plt.subplot(gs[1], sharex = ax)
        ax2.plot(df['time'],df['cumulative_voidage'],c='black')
        ax2.plot(df['time'],df['cumulative_aquifer'],c='blue')
        plt.grid(b=True, which='major', axis='both', color='grey', linestyle='-', alpha = 0.5)
        
        # add label for y axis
        plt.ylabel('Cumulative offtake/aquifer (mmrb)')

    # set x limits (future option to change the start date ...)
    plt.xlim(pd.to_datetime(t_min),pd.to_datetime(t_max))
    
    # display the plot
    plt.show()

In [ ]:
def most_recent_pres(PRES_data_all):

    # make deep copy of df
    df = PRES_data_all.copy(deep=True)

    # slice df on most recent build-up, producers only
    list_of_wells = df['Well'].unique()
    list_of_producers = []
    for well in d_well_dictionary.keys():
        if well in list_of_wells:
            if well[1] == 'P':
                list_of_producers.append(well)
    
    lastp_list = []
    for producer in list_of_producers:
        # slice df to that well only
        df_well = df[df['Well']==producer]
        # find the index corresponding to the last point in time
        list_of_times = df_well['time'].tolist()
        final_time_index = list_of_times.index(max(list_of_times))
        # print values
        final_p_values = df_well.iloc[final_time_index]       
        # append to list
        lastp_list.append([producer,d_well_dictionary[producer][i_dict_well_rmu], \
            (final_p_values['Datum_PRES_psi']-d_well_dictionary[producer][i_dict_well_datum])/bar_to_psi, \
            (final_p_values['Datum_PRES_psi_corrected']-d_well_dictionary[producer][i_dict_well_datum])/bar_to_psi, \

            (final_p_values['Datum_PRES_psi']-(d_well_dictionary[producer][i_dict_well_datum])+(d_well_dictionary[producer][i_dict_well_topcomp]))/bar_to_psi, \
            (final_p_values['Datum_PRES_psi_corrected']-(d_well_dictionary[producer][i_dict_well_datum])+(d_well_dictionary[producer][i_dict_well_topcomp]))/bar_to_psi, \
            (pd.Timestamp.now()-final_p_values['time']).days, \
            (final_p_values['Datum_PRES_psi'] - d_RMU_dictionary[d_well_dictionary[producer][i_dict_well_rmu]][i_dict_rmuwell_RDOL][i_dict_LNOL])/bar_to_psi, \
            final_p_values['Datum_PRES_psi'], \
            final_p_values['Datum_PRES_psi_corrected'], \
            final_p_values['time'], \
            final_p_values['PBU_PFO_type']])
    #
    # write series to df
    df_lastp = pd.DataFrame(lastp_list)
    df_lastp.columns = ['Well','RMU','PRES_gauge_bar','PRES_gauge_bar_corrected','PRES_topcomp_bar','PRES_topcomp_bar_corrected','time_since_PBU','Gap_to_LNOL_bar','PRES_datum_psi','PRES_datum_corrected_psi','PBU_datetime','PBU_type']

    return df_lastp.sort_values(by='Well')

In [ ]:
def quad_rdol_plotter(PRES_data_all, Forecast_RMU_rollup):
    #
    # Plots Schiehallion and Loyal RMUs
    #
    # For various plot options, go to the global code
    # Contact: D Robbins, RE, WoS
    #          T Harpley, RE, WoS
    #
    # take a deep copy of the input df so as not to mess it up
    df = PRES_data_all.copy(deep=True)
    rmu_df = Forecast_RMU_rollup.copy(deep=True)
    #
    # plot options
    subset_chosen = 'Loyal'         # subset of RMUs to plot
    bool_plot_well_annotations = False   # plot well annotations
    bool_rdol_latest = False             # plot the latest pressure
    bool_injectors = False              # plot the last flowing pressure of injectors (not recommended right now)
    bool_plot_consistent = False         # current and previous PRES pressures are both from t+dt buildups only
    bool_plot_corrected = False     # if true, plot corrected pressure
    bool_plot_psat_lines = False
    bool_plot_setpoint = True
    bool_plot_voidage = True
    #
    # set up plot options (note size ratio is different depending on the number of RMUs in subset ...)
    if subset_chosen == 'Schiehallion':
        fig = plt.figure(figsize=(15,13))
        n_cols_legend = 3
        x_label_tick_size = 'medium'
        y_label_tick_size = 'medium'
        y_axis_label_size = 'large'
        legend_label_tick_size = 'large'
        annotation_label_size = 'medium'
        title_size = 'xx-large'
    elif subset_chosen == 'Loyal': 
        fig = plt.figure(figsize=(7,13))
        n_cols_legend = 2
        x_label_tick_size = 'medium'
        y_label_tick_size = 'medium'
        y_axis_label_size = 'large'
        legend_label_tick_size = 'medium'
        annotation_label_size = 'medium'
        title_size = 'xx-large'
    else:
        fig = plt.figure(figsize=(18,13))
        n_cols_legend = 3
        x_label_tick_size = 'medium'
        y_label_tick_size = 'medium'
        y_axis_label_size = 'large'
        legend_label_tick_size = 'large'
        annotation_label_size = 'medium'
        title_size = 'xx-large'
    #
    # if set points are plot, don't plot saturation pressure lines as these are already incorporated
    if bool_plot_setpoint == True:
        bool_plot_psat_lines == False

    # general plot options
    if bool_plot_voidage == True:
        gs = gridspec.GridSpec(2, 1, height_ratios=[4, 1])
        gs.update(hspace=0.1)
        ax = plt.subplot(gs[0])
    #
    # build the well/RMU lists
    if subset_chosen == 'Schiehallion':
        RMU_names = [rmu for rmu in list_of_RMUs if "Loy" not in rmu and "All" not in rmu]
    elif subset_chosen == 'Loyal':
        RMU_names = [rmu for rmu in list_of_RMUs if "Loy" in rmu or "All" in rmu]
    else:
        RMU_names = list_of_RMUs
    
    # pop out NWAD
    try:
        RMU_names.remove('NWAD')
    except:
        pass

    # build x_range
    x_range = [i for i in range(0,len(RMU_names))]

    # chop IPC/IIC dataframe to give RMU subset
    rmu_df = rmu_df[rmu_df['RMU'].isin(RMU_names)]

    # get the list of wells
    well_list = []
    for well in [str for str in d_well_dictionary.keys()]:
        if d_well_dictionary[well][i_dict_well_rmu] in RMU_names:
            well_list.append(well)
    #
    ##########################
    # RDOL bar chart plotter #
    ##########################
    # loop iterators: i refers to RMU order, j is local loop within this loop
    #
    for i,RMU in enumerate(RMU_names):
        #
        # pull list from global variables and make a deep copy of this so that the global variable remains unharmed
        RDOL_absolute = d_RMU_dictionary[RMU][i_dict_rmuwell_RDOL]
        # append the USOL to give some 'redspace' at the top
        RDOL_absolute.append(RDOL_absolute[len(RDOL_absolute)-1]+upper_buffer)
        # delta the limits so we can plot a stacked bar chart
        RDOL_deltas = [RDOL_absolute[0]]
        for j in range(1,len(RDOL_absolute)):
            RDOL_deltas.append(RDOL_absolute[j]-RDOL_absolute[j-1])
        # now run the plotter
        for j in range(0,len(RDOL_deltas)):
            if j == 0:            # first element isn't stacked
                ax.bar(x_range[i],RDOL_deltas[j],color=rdol_colour_map[j],edgecolor='black')
            else:                 # other elements are stacked so need a bottom= reference
                ax.bar(x_range[i],RDOL_deltas[j],bottom=RDOL_absolute[j-1],color=rdol_colour_map[j],edgecolor='black')
        #
    #
    ###############################################
    # Two most recent reservoir pressures plotter #
    ###############################################
    #
    i_prod = 0
    i_prod_latest = 0
    i_inj = 0
    #
    for i,well in enumerate(well_list):
        #
        # grab well type
        well_type = d_well_dictionary[well][i_dict_well_type]
        #
        # set up the x-axis (RMU) point for this well
        well_RMU = d_well_dictionary[well][i_dict_well_rmu]
        x_scatter_jitter = x_range[RMU_names.index(well_RMU)] + np.random.randint(-1*jitter_magnitude,jitter_magnitude)*jitter_strength
        #
        # set up the y-axis (pressure) points for this well
        # note we want to ignore "final" buildup types as these are not the usual consistent ones
        if bool_plot_consistent == True:
            list_of_times = df[(df['Well']==well) & (df['PBU_PFO_type']==pres_type_standard)]['time'].to_list()
            if bool_plot_corrected == True:
                list_of_pressures = df[(df['Well']==well) & (df['PBU_PFO_type']==pres_type_standard)]['Datum_PRES_psi_corrected'].to_list()
            else:
                list_of_pressures = df[(df['Well']==well) & (df['PBU_PFO_type']==pres_type_standard)]['Datum_PRES_psi'].to_list()
            #
            # grab list of PBU/PFO types (should all be the same!)
            list_of_pressure_types = df[(df['Well']==well) & (df['PBU_PFO_type']==pres_type_standard)]['PBU_PFO_type'].to_list()
            #
        else:
            list_of_times = df[(df['Well']==well) & (df['PBU_PFO_type'] != pres_type_final) & (df['PBU_PFO_type'] != pres_type_lastpoint)]['time'].to_list()
            if bool_plot_corrected == True:
                list_of_pressures = df[(df['Well']==well) & (df['PBU_PFO_type'] != pres_type_final) & (df['PBU_PFO_type'] != pres_type_lastpoint)]['Datum_PRES_psi_corrected'].to_list()
            else:
                list_of_pressures = df[(df['Well']==well) & (df['PBU_PFO_type']!= pres_type_final) & (df['PBU_PFO_type'] != pres_type_lastpoint)]['Datum_PRES_psi'].to_list()
            #
            # get list of PBU / PFO types
            list_of_pressure_types = df[(df['Well']==well) & (df['PBU_PFO_type']!= pres_type_final) & (df['PBU_PFO_type'] != pres_type_lastpoint)]['PBU_PFO_type'].to_list()
        #
        # if a well doesn't have any data points, don't try to do any plotting (e.g. new well or only one PBU)
        if (len(list_of_pressures) > 1) and (len(list_of_times) > 1):
            y_pressure = list_of_pressures[list_of_times.index(max(list_of_times))]  
            y_pressure_old = list_of_pressures[list_of_times.index(max(n for n in list_of_times if n!= list_of_times[list_of_times.index(max(list_of_times))]))]
            
            # assign each type of pressure build-up
            y_pressure_type = list_of_pressure_types[list_of_times.index(max(list_of_times))]  
            y_pressure_old_type = list_of_pressure_types[list_of_times.index(max(n for n in list_of_times if n!= list_of_times[list_of_times.index(max(list_of_times))]))]

            # apply the correct marker type to each pressure type
            if y_pressure_type == pres_type_final:
                y_pressure_marker = 'x'
            elif y_pressure_type == pres_type_manual:
                y_pressure_marker = '+'
            else:
                y_pressure_marker = 'o'
            #
            if y_pressure_old_type == pres_type_final:
                y_pressure_old_marker = 'x'
            elif y_pressure_old_type == pres_type_manual:
                y_pressure_old_marker = '+'
            else:
                y_pressure_old_marker = 'o'

            # grab the y data for the last build-up pressure, just in case (should just be one value!)
            i_latest_available = 0
            try:
                if bool_plot_corrected == True: # right now, this should be the same
                    y_pressure_latest = df[(df['Well']==well) & (df['PBU_PFO_type']==pres_type_final)]['Datum_PRES_psi_corrected'].values[0]
                else:
                    y_pressure_latest = df[(df['Well']==well) & (df['PBU_PFO_type']==pres_type_final)]['Datum_PRES_psi'].values[0]
                i_latest_available = 1
            except: 
                y_pressure_latest = 0.0
            #
            # scatter plot of (x,y) value found
            if well_type == 'prod':
                if i_prod == 0: # set up label (there's probably a much less verbose way to do this, but that's how I roll / rolled)
                    i_prod += 1
                    #
                    # Most recent reservoir pressure
                    if y_pressure_type == pres_type_longer: # separate hollow marker for long shut-ins (short-term are filled)
                        ax.scatter(x_scatter_jitter,y_pressure,s=40,zorder=2,label='Latest reservoir pressure',marker=y_pressure_marker,facecolors='none',edgecolors='red')
                    else:
                        ax.scatter(x_scatter_jitter,y_pressure,s=40,c='red',zorder=2,label='Latest reservoir pressure',marker=y_pressure_marker)
                    #
                    # Penultimate reservoir pressure
                    if y_pressure_old_type == pres_type_longer: # separate hollow marker for long shut-ins (short-term are filled)
                        ax.scatter(x_scatter_jitter,y_pressure_old,s=20,zorder=2,label='Penultimate reservoir pressure',marker=y_pressure_old_marker,facecolors='none',edgecolors='black')
                    else:
                        ax.scatter(x_scatter_jitter,y_pressure_old,s=20,c='black',zorder=2,label='Penultimate reservoir pressure',marker=y_pressure_old_marker)
                    #
                    if bool_rdol_latest == True:
                        if i_latest_available == 1: # may not have had long enough shut in last time
                            i_prod_latest += 1
                            ax.scatter(x_scatter_jitter,y_pressure_latest,s=40,zorder=2,facecolors='none',edgecolors='blue',label='Latest build-up pressure',marker='x')
                    #
                else:
                    #
                    # Latest reservoir pressure
                    if y_pressure_type == pres_type_longer: # separate hollow marker for long shut-ins (short-term are filled)
                        ax.scatter(x_scatter_jitter,y_pressure,s=40,zorder=2,marker=y_pressure_marker,facecolors='none',edgecolors='red')
                    else:
                        ax.scatter(x_scatter_jitter,y_pressure,s=40,c='red',zorder=2,marker=y_pressure_marker)
                    #
                    # Previous reservoir pressure
                    if y_pressure_old_type == pres_type_longer: # separate hollow marker for long shut-ins (short-term are filled)
                        ax.scatter(x_scatter_jitter,y_pressure_old,s=20,zorder=2,marker=y_pressure_old_marker,facecolors='none',edgecolors='black')
                    else:
                        ax.scatter(x_scatter_jitter,y_pressure_old,s=20,c='black',zorder=2,marker=y_pressure_old_marker)
                    #
                    if bool_rdol_latest == True:
                        if i_latest_available == 1: # may not have had long enough shut in last time
                            if i_prod_latest == 0:
                                ax.scatter(x_scatter_jitter,y_pressure_latest,s=40,c='blue',zorder=2,label='Latest build-up pressure',marker='x')
                                i_prod_latest += 1
                            else:
                                ax.scatter(x_scatter_jitter,y_pressure_latest,s=40,c='blue',zorder=2,marker='x')
                    #
            else:
                plot_inj_label = 0
                if bool_injectors == True:
                    if i_inj == 0: # set up label
                        # if i_latest_available == 1:
                        ax.scatter(x_scatter_jitter,y_pressure,s=40,c='blue',zorder=2,label='Latest reservoir pressure',marker='o')
                        i_inj += 1
                        plot_inj_label = 1
                    else:
                        # if i_latest_available == 1:
                        plot_inj_label = 1
                        ax.scatter(x_scatter_jitter,y_pressure,s=40,c='blue',zorder=2,marker='o')
            #
            # now set up the annotation/labelling (most recent point only)
            # ... want to just offset this slightly so it doesn't cover the plotted point
            ha_lab = ha_label[np.random.randint(0,2)]
            va_lab = va_label[np.random.randint(0,2)]
            if ha_lab == 'left':
                x_ha_delta = 0.02
            else:
                x_ha_delta = -0.02
            if va_lab == 'top':
                y_va_delta = 0.02
            else:
                y_va_delta = -0.02
            if bool_plot_well_annotations == True:
                if well_type == 'prod':
                    plt.annotate(well,xy=(x_scatter_jitter,y_pressure),ha=ha_lab,va=va_lab,xytext=(x_scatter_jitter+x_ha_delta,y_pressure+y_va_delta),size=annotation_label_size)
                else:
                    if bool_injectors == True:
                        plt.annotate(well,xy=(x_scatter_jitter,y_pressure),ha=ha_lab,va=va_lab,xytext=(x_scatter_jitter+x_ha_delta,y_pressure+y_va_delta),size=annotation_label_size)
            #
            # for my last trick ... also plot directional arrow for pressure trend
            delta_p = y_pressure - y_pressure_old
            delta_p_current = y_pressure_latest - y_pressure
            # generate the origin of the arrow base
            arrow_x = x_scatter_jitter + 0.01
            arrow_y = y_pressure_old
            arrow_y_current = y_pressure
            # generate the arrow movement
            arrow_dx = 0.0
            arrow_dy = delta_p
            arrow_dy_current = delta_p_current
            # arrow options
            arrow_width = 0.01
            head_width = 20*arrow_width
            head_length = 400*head_width
            if delta_p > 0:
                arrow_color = 'green'
            elif delta_p < 0:
                arrow_color = 'red'
            else: # in the rare caase there is identically no change in pressure ...
                arrow_color = 'black'
            if d_well_dictionary[well][i_dict_well_type] == 'prod':
                plt.arrow(arrow_x,arrow_y,arrow_dx,arrow_dy,color=arrow_color,width=arrow_width)
                if bool_rdol_latest == True:
                    if i_latest_available == 1:
                        plt.arrow(arrow_x,arrow_y_current,arrow_dx,arrow_dy_current,color='blue',width=arrow_width)
            #
    #
    #########################################
    # Saturation / set point pressure lines #
    #########################################
    #
    if bool_plot_psat_lines == True:
        #
        # build list of initial saturation pressures per RMU
        psat0_yvals = []
        for i,RMU in enumerate(RMU_names):
            psat0_yvals.append(d_RMU_plotter_dictionary[RMU][i_dict_plotter_psat0][i_dict_plotter_value])
        ax.plot(x_range,psat0_yvals,c='purple',label='Initial saturation pressure')
        #
        # now build updated saturation pressures per RMU
        # bit more complicated for the other ones - assume that we have a max of two updated psats for now
        psat_yvals_1 = []
        psat_yvals_2 = []
        for i,RMU in enumerate(RMU_names):
            if len(d_RMU_plotter_dictionary[RMU]) == 3: # this is specified as psat if there is the lnol-ps0-ps standard dictionary
                psat_yvals_1.append(d_RMU_plotter_dictionary[RMU][i_dict_plotter_psat][i_dict_plotter_value])
                psat_yvals_2.append(d_RMU_plotter_dictionary[RMU][i_dict_plotter_psat][i_dict_plotter_value])
            elif len(d_RMU_plotter_dictionary[RMU]) == 2: # no updated psat values for Loyal
                psat_yvals_1.append(None)
                psat_yvals_2.append(None)
            else:                                          # else we have more than one updated psat for this RMU
                psat_yvals_1.append(d_RMU_plotter_dictionary[RMU][i_dict_plotter_psat][i_dict_plotter_value])
                psat_yvals_2.append(d_RMU_plotter_dictionary[RMU][i_dict_plotter_psat2][i_dict_plotter_value])
        #
        # now plot these updated saturation pressures
        # use a nice Scottish thistle colour for these :-)
        ax.plot(x_range,psat_yvals_1,c='thistle',label='Updated saturation pressure',linestyle='dashed')
        ax.plot(x_range,psat_yvals_2,c='thistle',linestyle='dashed')
    #
    # set point lines
    if bool_plot_setpoint == True:
        #
        # build list of set point values
        pset_low_vals = []
        pset_vals = []
        pset_high_vals = []
        #
        for i,RMU in enumerate(RMU_names):
            pset_low_vals.append(d_optimum_operating_point[RMU][i_optimum_low])
            pset_vals.append(d_optimum_operating_point[RMU][i_optimum_mid])
            pset_high_vals.append(d_optimum_operating_point[RMU][i_optimum_high])
        #
        # plot the set point
        ax.plot(x_range,pset_vals,c='forestgreen',label='RMU target set point')
        #
    #
    ################
    # Axes options #
    ################
    #
    # set up the x-axis labels
    #
    rmu_labels = []
    if plot_dictionary == 1: # generate the full segment name
        for RMU in RMU_names:
            rmu_labels.append(RMU_dictionary[RMU])
        # wrapping the labels creates nicer formatting on plot without needing to rotate axes
        rmu_labels = [ '\n'.join(wrap(l, 7)) for l in rmu_labels ]
    else:
        rmu_labels = RMU_names
    #
    # x axis options
    #
    plt.xticks(x_range,rmu_labels)
    plt.xlim(-1,len(x_range)) # creates a 1 unit buffer on either end
    plt.tick_params(
        axis='x',
        which='both',
        labelsize=x_label_tick_size)
    #
    # y axis options
    #
    ax.set_ylabel('Datumised pressure (psia)', labelpad=5, fontsize=y_axis_label_size)
    plt.ylim(ylim_lower,ylim_upper)
    plt.tick_params(
        axis='y',
        which='both',
        labelsize=y_label_tick_size)
    #
    # chart title
    #
    if subset_chosen == 'Schiehallion':
        plottitle = 'Schiehallion RDOL status'
    elif subset_chosen == 'Loyal': 
        plottitle = 'Loyal RDOL status'
    else:
        plottitle = 'Quad 204 RDOL status'
    plt.title(plottitle,fontsize=title_size,y=1.02)
    #
    # finally done!
    #
    plt.legend(loc='upper center',ncol=n_cols_legend,fontsize=legend_label_tick_size,shadow='bool')

    ##################################
    # Voidage Gaps bar chart plotter #
    ##################################

    if bool_plot_voidage == True:

        # start writing on new plot/grid space
        ax2 = plt.subplot(gs[1])

        # build list of IPC/IIC positions (loop this so order doesn't matter)
        y_IPC_IIC = []
        bar_colors = []
        for RMU in RMU_names:
            if RMU == 'NClawInj': # N Claw producer is referred to as "NClaw on RMU_list" so don't need special option for this
                y_IPC_IIC.append(0.0)
                bar_colors.append('limegreen')
            else:
                rmu_voidage_position = rmu_df[rmu_df['RMU']==RMU]['voidage_rbpd'].iloc[0]
                y_IPC_IIC.append(rmu_voidage_position*0.001) # voidage derived in rbpd, want to report mrb/d
                bar_colors.append('limegreen') if rmu_voidage_position > 0 else bar_colors.append('red')     

        # plot bar chart    
        ax2.bar(x_range, y_IPC_IIC, color=bar_colors)

        #titles
        ax2.set_ylabel('Forecast voidage gap (mrb/d)', labelpad=5, fontsize=y_axis_label_size)

        #x-axis options
        plt.xticks(x_range,rmu_labels)
        plt.xlim(-1,len(x_range))
        plt.tick_params(
            axis='x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom=False,      # ticks along the bottom edge are off
            top=False,         # ticks along the top edge are off
            labelbottom=False) # labels along the bottom edge are off
            
        #y-axis options
        plt.ylim(-50,50)
        plt.locator_params(axis='y', nbins=4)
        plt.tick_params(
            axis='y',
            which='both',
            labelsize=y_label_tick_size)

        #plot options
        ax2.grid(False)
        ax2.set_facecolor((1,1,1))
        ax2.spines['top'].set_visible(False)
        ax2.spines['right'].set_visible(False)
        ax2.spines['bottom'].set_visible(False)
        #ax2.spines['left'].set_visible(False)

        # add line at 0 on y-axis
        ax2.plot(range(-1,len(x_range)+1),[0.0] * (len(x_range)+2), color='black', linewidth=1)
        
        # Call the global function. All the magic happens there.
        add_value_labels(annotation_label_size,ax2)

    plt.show()